In [ ]:
#Alur
#Setup
#Buka File
#Lihat data
#Tentukan X dan y
#Cek missing value
#Handle missing value
#Feature Engineering
#Buat Model
#Cari RMSE paling kecil
#fit model ke data test
#Finishing

In [1]:
#Setup
import pandas as pd
import numpy as np

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

print("Setup Done")

Setup Done


In [80]:
#Import Data
data = pd.read_csv("C:/Users/visco/Downloads/playground-series-s6e1/train.csv")
tdata = pd.read_csv("C:/Users/visco/Downloads/playground-series-s6e1/test.csv")
print(data.shape)
print(data.columns)

(630000, 13)
Index(['id', 'age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'exam_score'],
      dtype='object')


In [3]:
#Lihat data
data.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [ ]:
mis = data.isnull().sum()
mis[0:13]

In [81]:
#cari kolom kategorikal untuk model catboost

for col in data:
    if data[col].dtype == "object":
        print(col)

cat_col= [
    'sleep_quality',
    'study_method',
    'facility_rating',
    'internet_access',
    'exam_difficulty'
]

gender
course
internet_access
sleep_quality
study_method
facility_rating
exam_difficulty


In [82]:
#tentukan X dan y
y = data["exam_score"]
X = data.drop(["id","exam_score","gender","course"], axis = 1)
# X = data.drop(["id","exam_score","internet_access","gender","exam_difficulty"], axis = 1)

In [14]:
#factorize category type columns
for col in X.select_dtypes("object"):
    X[col], _ = X[col].factorize()

fcol = X.dtypes ==int

In [16]:
#mi score
from sklearn.feature_selection import mutual_info_regression
def make_mi_scores(X,y, cat_col):
    mi_scores = mutual_info_regression(X, y, discrete_features= fcol)
    mi_scores = pd.Series(mi_scores, name= "MI Scores", index = X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X,y,fcol)
mi_scores[::]

study_hours         0.913950
class_attendance    0.301945
sleep_hours         0.073860
study_method        0.040896
sleep_quality       0.036648
facility_rating     0.030615
age                 0.011017
exam_difficulty     0.008920
internet_access     0.001987
Name: MI Scores, dtype: float64

In [ ]:
#buat model
model = CatBoostRegressor(
    iterations=1500, 
    learning_rate= 0.05,
    depth = 8,
    loss_function= 'RMSE',
    cat_features= cat_col,
    random_seed= 2)

In [84]:
#Train model
train_X, val_X, train_y, val_y = train_test_split(X,y, test_size= 0.2, random_state=1)
model.fit(train_X,train_y,)

0:	learn: 18.3129860	total: 256ms	remaining: 6m 24s
1:	learn: 17.7552058	total: 485ms	remaining: 6m 3s
2:	learn: 17.2337352	total: 764ms	remaining: 6m 21s
3:	learn: 16.7252469	total: 1.03s	remaining: 6m 24s
4:	learn: 16.2465379	total: 1.24s	remaining: 6m 9s
5:	learn: 15.7997363	total: 1.45s	remaining: 6m 1s
6:	learn: 15.3848309	total: 1.63s	remaining: 5m 47s
7:	learn: 14.9945224	total: 1.8s	remaining: 5m 35s
8:	learn: 14.6318461	total: 1.95s	remaining: 5m 22s
9:	learn: 14.2907061	total: 2.14s	remaining: 5m 19s
10:	learn: 13.9726175	total: 2.31s	remaining: 5m 12s
11:	learn: 13.6811969	total: 2.46s	remaining: 5m 5s
12:	learn: 13.4049017	total: 2.64s	remaining: 5m 1s
13:	learn: 13.1500944	total: 2.81s	remaining: 4m 58s
14:	learn: 12.8960889	total: 3s	remaining: 4m 57s
15:	learn: 12.6560863	total: 3.22s	remaining: 4m 58s
16:	learn: 12.4348525	total: 3.37s	remaining: 4m 54s
17:	learn: 12.2313435	total: 3.57s	remaining: 4m 53s
18:	learn: 12.0402376	total: 3.75s	remaining: 4m 51s
19:	learn: 1

In [85]:
#predict
preds_vy= model.predict(val_X) #predict jawaban ujian
preds_ty= model.predict(train_X) #predict jawaban latihan soal

In [86]:
#Model Validation
train_rmse = root_mean_squared_error(train_y, preds_ty)
val_rmse = root_mean_squared_error(val_y,preds_vy)
print(train_rmse)
print(val_rmse)

8.73302785876941
8.773170534050665


In [62]:
penting = model.get_feature_importance(prettified=True)
print(penting)

         Feature Id  Importances
0       study_hours    50.767137
1  class_attendance    18.595252
2      sleep_impact    13.680116
3      study_method     8.483585
4   facility_rating     6.839227
5       sleep_hours     0.595563
6       study_rasio     0.501639
7     sleep_quality     0.387743
8               age     0.132838
9   internet_access     0.016901


In [87]:
#fit model ke Semua data
model.fit(X,y)

0:	learn: 18.3173942	total: 236ms	remaining: 5m 53s
1:	learn: 17.7549377	total: 433ms	remaining: 5m 24s
2:	learn: 17.2327752	total: 645ms	remaining: 5m 21s
3:	learn: 16.7440207	total: 912ms	remaining: 5m 40s
4:	learn: 16.2632645	total: 1.13s	remaining: 5m 36s
5:	learn: 15.8171211	total: 1.32s	remaining: 5m 29s
6:	learn: 15.4009111	total: 1.53s	remaining: 5m 27s
7:	learn: 15.0110833	total: 1.75s	remaining: 5m 27s
8:	learn: 14.6473652	total: 1.98s	remaining: 5m 28s
9:	learn: 14.3064986	total: 2.17s	remaining: 5m 22s
10:	learn: 13.9891756	total: 2.37s	remaining: 5m 20s
11:	learn: 13.6935437	total: 2.59s	remaining: 5m 20s
12:	learn: 13.4171708	total: 2.84s	remaining: 5m 25s
13:	learn: 13.1566936	total: 3.02s	remaining: 5m 20s
14:	learn: 12.8984575	total: 3.24s	remaining: 5m 20s
15:	learn: 12.6601021	total: 3.43s	remaining: 5m 18s
16:	learn: 12.4357080	total: 3.62s	remaining: 5m 15s
17:	learn: 12.2322625	total: 3.81s	remaining: 5m 13s
18:	learn: 12.0384771	total: 3.98s	remaining: 5m 10s
19:

In [88]:
#Prediksi test data
tdata_fit= tdata.drop(["id","gender","course"], axis=1)
pred_test= model.predict(tdata_fit)

In [ ]:
print(X.columns)
print(tdata_fit.columns)

In [89]:
#Buat ke dataset
submit = pd.DataFrame({"id":tdata.id, "exam_score": pred_test})
#Buat dataset ke file
submit.to_csv("P1.csv", index= False)